# Assignment 3 - Deep Learning

Machine Learning (BBWL), Michael Mommert, FS2023, University of St. Gallen

The **goal** of this assignment is to implement and train a neural network to perform image classification. While a good performance of the resulting trained model is desirable, it is more important to follow the task setup carefully and implement your code following best practices.

The dataset used is [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), which consists of 32x32 RGB images, showing objects from either of 10 different classes. 

Your **objectives** are the following:
* Implement a neural network architecture with at least 6 layers for the task of image classification. You can use any architecture you like.
* For each training epoch, output the loss on the training dataset and the loss on the validation dataset. Tune the learning rate using this setup (only use full and half decimal powers, e.g., 0.001, 0.005, 0.01, 0.05, ...) to maximize the accuracy on the validation dataset and prevent overfitting. Visualize the training and validation loss as a function of epoch for the best-performing learning rate in the same plot.
* Evaluate your final trained and tuned model on the test dataset by computing accuracy, precision and recall, visualize the confusion matrix and discuss implications.

This assignment will be **graded** based on:
* whether these objectives have been achieved;
* whether the solution follows best practices;
* how well the approach is documented (e.g., using text cells, plots, etc.);
* how clean the code is.

There are no restrictions on the resources that you can use -- collaborating on assignments is allowed -- but students are not allowed to submit identical code.

There will be a leaderboard comparing the accuracies evaluated on the test dataset; the winner will receive a [grand prize](https://en.wikipedia.org/wiki/Mars_(chocolate_bar))!

Please submit your runnable Notebook to [michael.mommert@unisg.ch](mailto:michael.mommert@unisg.ch) **before 17 May 2023, 23:59**. Please include your name in the Notebook filename.

-----

The following code cells will setup the environment, download and prepare the data for you. Please do not modify these code cells.

In [ ]:
# import standard python libraries
from datetime import datetime
import numpy as np

# import the PyTorch deep learning libary
import torch, torchvision
import torch.nn.functional as F
from torch import nn, optim

# import sklearn classification evaluation library
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# import plotting capabilities
import matplotlib.pyplot as plt
import seaborn as sns


# init deterministic seed
seed_value = 42
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
torch.cuda.manual_seed(seed_value)
print('[LOG] notebook with {} computation enabled'.format(str(device)))

In [ ]:
# import the Google Colab GDrive connector
from google.colab import drive

# mount GDrive inside the Colab notebook
drive.mount('/content/drive')

# create data sub-directory inside the Colab Notebooks directory
data_directory = '/content/drive/MyDrive/Colab Notebooks/data_fmnist'
if not os.path.exists(data_directory): os.makedirs(data_directory)

In [ ]:
# download training images and split data (X) from labels (y)
train_path = data_directory + '/train_cifar10'
cifar10_train = torchvision.datasets.CIFAR10(root=train_path, train=True, download=True)
X_train = cifar10_train.data
y_train = cifar10_train.targets

# download evaluation images and split into val and test datasets
eval_path = data_directory + '/eval_cifar10'
cifar10_eval = torchvision.datasets.CIFAR10(root=eval_path, train=False, download=True)
X_val, X_test, y_val, y_test = train_test_split(cifar10_eval.data, cifar10_eval.targets, test_size=0.5, stratify=cifar10_eval.targets, random_state=seed_value)

# define class names
cifar10_classes = cifar10_train.classes

print('Train: {}, Val: {}, Test: {}'.format(len(X_train), len(X_val), len(X_test)))

----

In [ ]:
# your code goes here